In [1]:
glove_file_path = "../data/glove.840B.300d.txt"
squad_file_path = '../data/train-v1.1.json'

In [2]:
import sys

In [3]:
sys.path.append('../')

In [4]:
from src.dataset import SquadDataset
from src.preprocessing import Preprocessing

In [5]:
print(sys.path)

['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\jetze\\.ipython', '../']


In [6]:
d = SquadDataset(squad_file_path, glove_file_path)

TypeError: __init__() missing 1 required positional argument: 'target'

In [ ]:
len(squad_dataset)